In [56]:
HOST, ADMIN, PASS, DB =  'mivo-db.cj2ucwgierrs.us-east-1.rds.amazonaws.com', 'admin', '58#64KDashz^bLrqTG2', 'mivo'

In [3]:
import pandas as pd

dtype_spec = {
    'Tipo': 'int',
    'Numero': 'int',
    'Lin': 'int',
    'C.Bod': 'int',
    'Cant': 'int',
    'Venta': 'int',
    'Costo': 'int',
    'Margen': 'int',
    'Jerarquia': str,
    'Sub Jerarquia': str,
    'Local': str,
    'Color': str,
    'talla': str,
    'Marca': str,
    'temporada': str,
    'Rut': str,
    'Cliente': str
}
df = pd.read_csv('Ventas.csv', usecols=list(dtype_spec.keys()) + ['Fecha'], dtype=dtype_spec)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y%m%d')

In [23]:
df['Local'].value_counts()

Local
INTERNET                          73734
SANTOS DUMONTT                    57076
DARDIGNAC                         54580
BUENOS AIRES                      35024
BODEGA LDP                        26897
CONCEPCION                        21063
VALDIVIA                          14119
MALL TOBALABA PTE.ALTO            12252
APUMANQUE                         10110
PROVIDENCIA                        8777
ESTADO                             6994
MALL QUILIN                        1990
MAYORISTA                           486
BODEGA DOMINICA                     449
Name: count, dtype: int64

In [32]:
flujo = pd.read_csv('Flujo.csv')
flujo['FECHA'] = pd.to_datetime(flujo['FECHA'], format='%m/%d/%Y')
flujo['Total Visitas'] = flujo.iloc[:, 2:26].sum(axis=1)
flujoSimple = flujo[['FECHA','ENTRADA', 'Total Visitas']]
flujoSimple.head()

,FECHA,ENTRADA,Total Visitas
0,2024-04-01,Concepcion,489
1,2024-04-02,Concepcion,362
2,2024-04-03,Concepcion,480
3,2024-04-04,Concepcion,586
4,2024-04-05,Concepcion,678


In [20]:
# Define data types for the hourly columns from 00 to 23
hour_columns_dtypes = {f"{hour:02}": int for hour in range(24)}

# 'ENTRADA' is specified as a string
dtype_dict = {
    'ENTRADA': str,
    **hour_columns_dtypes  # Merge the dictionary with hourly data types
}

# Load the CSV file with defined data types and specify 'FECHA' as a date column
traffic_df = pd.read_csv('Flujo.csv', dtype=dtype_dict, parse_dates=['FECHA'])

# Display the first few entries to check data loading
print(traffic_df.head())

# Check the data types to confirm proper loading and type assignment
#print(traffic_df.dtypes)

       FECHA     ENTRADA  0  1  2  3  4  5  6  7  ...  14  15  16  17  18  19  \
0 2024-04-01  Concepcion  0  0  0  0  0  0  0  0  ...  47  44  52  53  60  48   
1 2024-04-02  Concepcion  0  0  0  0  0  0  0  0  ...  22  25  33  57  34  33   
2 2024-04-03  Concepcion  0  0  0  0  0  0  0  0  ...  39  70  49  53  54  20   
3 2024-04-04  Concepcion  0  0  0  0  0  0  0  0  ...  81  47  60  51  63  33   
4 2024-04-05  Concepcion  0  0  0  0  0  0  0  0  ...  68  60  83  83  84  54   

   20  21  22  23  
0   1   0   0   0  
1   1   0   0   0  
2   1   0   0   0  
3   0   0   0   0  
4   1   0   0   0  

[5 rows x 26 columns]


In [7]:
import pandas as pd

# Assuming 'df' is already loaded and contains your sales data

# Ensure 'Fecha' is a datetime type for proper grouping
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Group by 'Fecha' and 'Local', and count distinct 'Numero'
daily_sales = df.groupby(['Fecha', 'Local']).agg(purchases=('Numero', 'nunique')).reset_index()

# Display the resulting DataFrame
print(daily_sales)

          Fecha                           Local  purchases
0    2024-01-02  APUMANQUE                              56
1    2024-01-02  BUENOS AIRES                           27
2    2024-01-02  CONCEPCION                            133
3    2024-01-02  DARDIGNAC                              36
4    2024-01-02  ESTADO                                 42
...         ...                             ...        ...
1263 2024-04-29  MALL TOBALABA PTE.ALTO                 37
1264 2024-04-29  MAYORISTA                              16
1265 2024-04-29  PROVIDENCIA                            44
1266 2024-04-29  SANTOS DUMONTT                         85
1267 2024-04-29  VALDIVIA                               49

[1268 rows x 3 columns]


In [31]:
visits = pd.DataFrame(flujoSimple)
sales = pd.DataFrame(daily_sales)

# Normalize the date columns
visits['FECHA'] = pd.to_datetime(visits['FECHA'])
sales['Fecha'] = pd.to_datetime(sales['Fecha'])


# Normalize the local/entrada names to lowercase
visits['ENTRADA'] = visits['ENTRADA'].astype(str).str.lower().str.strip()
sales['Local'] = sales['Local'].astype(str).str.lower().str.strip()

# Rename columns for consistency
visits.rename(columns={'FECHA': 'date', 'ENTRADA': 'local'}, inplace=True)
sales.rename(columns={'Fecha': 'date', 'Local': 'local'}, inplace=True)

# Merge DataFrames
result = pd.merge(visits, sales, on=['date', 'local'], how='inner')
result['conversion_rate'] = result['purchases'] / result['Total Visitas'] * 100
result.head()

,date,local,Total Visitas,purchases,conversion_rate
0,2024-04-01,concepcion,489,55,11.247444
1,2024-04-02,concepcion,362,55,15.193370
2,2024-04-03,concepcion,480,70,14.583333
3,2024-04-04,concepcion,586,93,15.870307
4,2024-04-05,concepcion,678,98,14.454277


In [8]:
# Drop redundant columns
merge_traffic_sales.drop(columns=['Fecha', 'Local'], inplace=True)

# Calculate the conversion rate and add it as the last column
merge_traffic_sales['conversion_rate'] = (merge_traffic_sales['purchases'] / merge_traffic_sales['Total Daily Entries'] * 100)

# Reorder columns if needed to put 'conversion_rate' last
merge_traffic_sales = merge_traffic_sales[['FECHA', 'ENTRADA', 'Total Daily Entries', 'purchases', 'conversion_rate']]

# Display the adjusted DataFrame
print(merge_traffic_sales)

NameError: name 'merge_traffic_sales' is not defined

### Conversion

In [41]:
import pandas as pd
dtype_spec = {
    'Tipo': 'int',
    'Numero': 'int',
    'Lin': 'int',
    'C.Bod': 'int',
    'Cant': 'int',
    'Venta': 'int',
    'Costo': 'int',
    'Margen': 'int',
    'Jerarquia': str,
    'Sub Jerarquia': str,
    'Local': str,
    'Color': str,
    'talla': str,
    'Marca': str,
    'temporada': str,
    'Rut': str,
    'Cliente': str
}
ventas = pd.read_csv('Ventas.csv', usecols=list(dtype_spec.keys()) + ['Fecha'], dtype=dtype_spec)
ventas['Fecha'] = pd.to_datetime(ventas['Fecha'], format='%Y%m%d')
daily_sales = ventas.groupby(['Fecha', 'Local']).agg(purchases=('Numero', 'nunique')).reset_index()
# clients_sales = ventas.groupby(['Cliente']).agg(
#     Total_Venta=('Venta', 'sum'),
#     Item_Count=('Numero', 'nunique'),
#     ).reset_index()


flujo = pd.read_csv('Flujo.csv')
flujo['FECHA'] = pd.to_datetime(flujo['FECHA'], format='%m/%d/%Y')
flujo['Total Visitas'] = flujo.iloc[:, 2:26].sum(axis=1)
flujoSimple = flujo[['FECHA','ENTRADA', 'Total Visitas']]

visits = pd.DataFrame(flujoSimple)
sales = pd.DataFrame(daily_sales)

# Normalize the date columns
visits['FECHA'] = pd.to_datetime(visits['FECHA'])
sales['Fecha'] = pd.to_datetime(sales['Fecha'])


# Normalize the local/entrada names to lowercase
visits['ENTRADA'] = visits['ENTRADA'].astype(str).str.lower().str.strip()
sales['Local'] = sales['Local'].astype(str).str.lower().str.strip()

# Rename columns for consistency
visits.rename(columns={'FECHA': 'date', 'ENTRADA': 'local'}, inplace=True)
sales.rename(columns={'Fecha': 'date', 'Local': 'local'}, inplace=True)

# Merge DataFrames
result = pd.merge(visits, sales, on=['date', 'local'], how='inner')
result['conversion_rate'] = result['purchases'] / result['Total Visitas'] * 100
result.head()

,date,local,Total Visitas,purchases,conversion_rate
0,2024-04-01,concepcion,489,55,11.247444
1,2024-04-02,concepcion,362,55,15.193370
2,2024-04-03,concepcion,480,70,14.583333
3,2024-04-04,concepcion,586,93,15.870307
4,2024-04-05,concepcion,678,98,14.454277


### Insert sales to DB [WORKING]

In [68]:
import pandas as pd
import pymysql

dtype_spec = {
    'Tipo': 'int',
    'Numero': 'int',
    'Lin': 'int',
    'C.Bod': 'int',
    'Cant': 'int',
    'Venta': 'int',
    'Costo': 'int',
    'Margen': 'int',
    'Jerarquia': str,
    'Sub Jerarquia': str,
    'Local': str,
    'Color': str,
    'talla': str,
    'Marca': str,
    'temporada': str,
    'Rut': str,
    'Cliente': str
}
ventas = pd.read_csv('Ventas.csv', usecols=list(dtype_spec.keys()) + ['Fecha'], dtype=dtype_spec)
ventas['Fecha'] = pd.to_datetime(ventas['Fecha'], format='%Y%m%d')
daily_sales = ventas.groupby(['Fecha', 'Local']).agg(purchases=('Numero', 'nunique')).reset_index()


sales = pd.DataFrame(daily_sales)


## Only some stores
salesFiltered = pd.DataFrame(sales)

# Mapping dictionary
local_map = {
    'concepcion': 1,
    'santos dumontt': 2,
    'mall tobalaba pte.alto': 3,
    'mall quilin': 4
}

# Apply the map to the 'local' column
salesFiltered['store_id'] = salesFiltered['local'].map(local_map).astype('Int64')

# Filter out rows where 'mapped_value' is None
filtered_sales = salesFiltered.dropna(subset=['store_id'])

# Print the filtered DataFrame
filtered_sales.head()



conn = pymysql.connect(host=HOST, user=ADMIN, password=PASS, database=DB)
cursor = conn.cursor()

insert_query = """
INSERT INTO sales (purchases, date, store_id)
VALUES (%s, %s, %s);
"""

# Iterate over DataFrame rows
for index, row in filtered_sales.iterrows():
    # Here you might add checks or transformations if needed
    # Prepare data for insertion
    data_to_insert = (row['purchases'], row['date'], row['store_id'])
    
    # Execute the SQL command
    try:
        cursor.execute(insert_query, data_to_insert)
        conn.commit()  # Commit changes for the current row only
    except pymysql.Error as e:
        print(f"Error on row {index}: {e}")
        conn.rollback()  # Roll back the current row insertion only
        continue  # Continue with the next row

# Close the connection
cursor.close()
conn.close()

### Insert visits-flujo to DB working

In [82]:
tobalaba = pd.read_csv('tobalaba_flujo.csv')
conce = pd.read_csv('conce_flujo.csv')
quilin = pd.read_csv('quilin_flujo.csv')
tobalaba['FECHA'] = pd.to_datetime(tobalaba['FECHA'], format='%Y-%m-%d')
conce['FECHA'] = pd.to_datetime(conce['FECHA'], format='%Y-%m-%d')
quilin['FECHA'] = pd.to_datetime(quilin['FECHA'], format='%Y-%m-%d')

conce['store_id'] = 1
tobalaba['store_id'] = 3
quilin['store_id'] = 4


total_visits = pd.concat([conce, tobalaba, quilin], ignore_index=True)

# Connect to the database
conn = pymysql.connect(host=HOST, user=ADMIN, password=PASS, database=DB)
cursor = conn.cursor()

# SQL Insert Statement
insert_query = """
INSERT INTO visits (count, time, date, store_id)
VALUES (%s, %s, %s, %s);
"""

# Iterate over DataFrame rows
for index, row in total_visits.iterrows():
    # Convert date to datetime format
    date = pd.to_datetime(row['FECHA']).date()  # Ensuring date is in proper format
    store_id = row['store_id']
    
    # Iterate over each hour's data column
    for hour in range(24):
        hour_key = f'{hour:02}'  # Format hour as two digits
        count = row[hour_key]
        if count > 0:
            time = f'{hour_key}:00:00'  # Format time as HH:MM:SS
            
            # Execute the SQL command
            try:
                cursor.execute(insert_query, (count, time, date, store_id))
                conn.commit()  # Commit changes for the current row only
            except pymysql.Error as e:
                print(f"Error on row {index} at hour {hour_key}: {e}")
                conn.rollback()  # Roll back the current row insertion only
                continue  # Continue with the next hour

# Close the connection
cursor.close()
conn.close()